In [2]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import random
import csv
import xgboost as xgb

In [ ]:
usecol_orig=['Id','L1_S24_F1559', 'L3_S32_F3851','L1_S24_F1827', 'L1_S24_F1582','L3_S32_F3854', 'L1_S24_F1510','L1_S24_F1525']

In [ ]:
# initial exploration

In [8]:
#n = 1183747 #number of records in file
def readrandom(path_ctg, path_num, numrow):
    X_getname = pd.read_csv(path_ctg,nrows=1)
    cols=X_getname.columns.tolist()  #column names must be the same
    
    X_getrows = pd.read_csv(path_ctg,usecols=[0])
    n= X_getrows.shape[0]            #total number of rows 
    skip = sorted(random.sample(range(n),n-numrow))

    X_train_ctg = pd.read_csv(path_ctg,skiprows=skip)
    X_train_ctg.columns=cols
    
    new_X_train_ctg=X_train_ctg.dropna(axis=1,how='all')
    
    print(new_X_train_ctg.shape)
    
    pd_ctg_rd=pd.get_dummies(new_X_train_ctg)
    
    response=pd.read_csv(path_num,usecols=[0],skiprows=skip)
    
    return pd_ctg_rd, response
    

In [3]:
def ctg_importance(X_tr,response):
    grid = {#'max_depth':10,
    #'booster': 'gbtree',
    'objective': 'binary:logistic',
    'subsample': 0.8,
    'colsample_bytree':0.8,
    'learning_rate': 0.1,
    'max_depth': 8,
    'seed': 2016,
    'silent': 0,
    'gamma':0.05}

    clf_ctg = xgb.XGBClassifier()
    clf_ctg.set_params(**grid)

    clf_ctg.fit(X_tr,response.iloc[:,0])
    
    column_name=X_tr.columns
    
    # get feature importance
    importance=clf_ctg.feature_importances_
    # sort index by importance value 
    importance_s=sorted([(index,value) for index, value in enumerate(importance.tolist())],key=lambda x: x[1],reverse = True)
    # filter
    important_feature_index=[index for (index,val) in importance_s if val>0.005]
    # get column name(full dummy name)
    col_name = [column_name[index] for index in important_feature_index]
    
    mid=[ele.split('_')[:3] for ele in col_name if '_' in ele]    
    # get original name
    imp_col_name=[]
        
    for ele in mid:
        if '_'.join(ele) in imp_col_name:
            continue
        else:
            imp_col_name.append('_'.join(ele))
            
    return imp_col_name

In [2]:
#n = 1183747 #number of records in file
def readchunk(path_ctg, path_num):
    size=250000
    for tr, res in zip(pd.read_csv(path_ctg, chunksize=size), 
                       pd.read_csv(path_num, usecols=['Response'], chunksize=size)):
        new_X_train_ctg=tr.dropna(axis=1,how='all')
        pd_ctg=pd.get_dummies(new_X_train_ctg)
        
        cols=ctg_importance(pd_ctg, res)
        print(cols)
    
#   new_X_train_ctg=X_train_ctg.dropna(axis=1,how='all')
        

    

In [4]:
def main():
    path_ctg='Y://Data//Kaggle//bosch//train_categorical//train_categorical.csv'
    path_num='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'
    numrow=300000
    X_train, Y_train= readrandom(path_ctg,path_num, numrow)
    selected_feature=ctg_importance(X_train, Y_train)
    print(selected_feature)
    return selected_feature

In [4]:
def main_1():
    path_ctg='Y://Data//Kaggle//bosch//train_categorical//train_categorical.csv'
    path_num='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'

    readchunk(path_ctg, path_num)

In [9]:
#0.005
l1=['L3_S29_F3317', 'L2_S26_F3038', 'L3_S29_F3475', 'L1_S24_F1584', 'L2_S26_F3099', 'L2_S27_F3131', 'L1_S24_F1510', 
    'L3_S32_F3851', 'L2_S27_F3192', 'L3_S32_F3854', 'L1_S24_F1675', 'L1_S24_F1559', 'L1_S25_F2141', 'L1_S24_F1523', 
    'L3_S35_F3899', 'L3_S29_F3320', 'L1_S24_F1537', 'L1_S25_F1852', 'L1_S24_F1585', 'L1_S25_F1981', 'L3_S47_F4141']

l2=['L3_S29_F3317', 'L2_S26_F3038', 'L3_S29_F3475', 'L2_S27_F3131', 'L3_S32_F3851', 'L2_S26_F3099', 'L3_S32_F3854',
    'L1_S24_F1584', 'L1_S24_F1675', 'L2_S27_F3192', 'L1_S25_F1981', 'L1_S24_F1559', 'L3_S47_F4141', 'L1_S24_F1510',
    'L1_S25_F1852', 'L2_S26_F3042', 'L3_S29_F3320', 'L3_S35_F3899', 'L2_S28_F3224', 'L3_S49_F4217']

l3=['L3_S29_F3317', 'L2_S26_F3038', 'L3_S29_F3475', 'L2_S27_F3131', 'L2_S26_F3099', 'L3_S32_F3854', 'L3_S32_F3851',
    'L1_S24_F1584', 'L2_S27_F3192', 'L1_S24_F1675', 'L1_S24_F1510', 'L1_S24_F1187', 'L1_S24_F1559', 'L1_S24_F1585',
    'L1_S25_F2141', 'L1_S25_F1981', 'L1_S24_F1525', 'L1_S25_F2099', 'L1_S24_F1114', 'L1_S24_F1523', 'L1_S25_F1852', 
    'L1_S25_F1903', 'L3_S47_F4141']

l4=['L3_S29_F3317', 'L3_S29_F3475', 'L2_S26_F3038', 'L2_S27_F3131', 'L3_S32_F3854', 'L1_S24_F1584', 'L3_S32_F3851', 
    'L2_S26_F3099', 'L2_S27_F3192', 'L1_S24_F1523', 'L1_S24_F1675', 'L1_S25_F2141', 'L1_S24_F675', 'L2_S26_F3042', 
    'L1_S24_F1278', 'L1_S25_F1852', 'L1_S24_F695', 'L1_S24_F1187', 'L1_S24_F1510', 'L1_S25_F1903', 'L1_S25_F2802']

l5=['L3_S29_F3317', 'L3_S29_F3475', 'L2_S26_F3038', 'L2_S27_F3131', 'L3_S32_F3854', 'L2_S26_F3099', 'L2_S27_F3192', 
    'L3_S32_F3851', 'L1_S24_F1584', 'L1_S24_F1510', 'L1_S24_F1582', 'L1_S25_F2141', 'L1_S24_F1559', 'L1_S25_F1903',
    'L1_S24_F1675', 'L1_S25_F3013', 'L1_S24_F819', 'L1_S25_F1852', 'L1_S24_F1585', 'L3_S29_F3320']

l005=list(set(l1) | set(l2) | set(l3) | set(l4) | set(l5))
print(l005)
print(len(l005))

['L2_S26_F3038', 'L3_S32_F3851', 'L1_S24_F1582', 'L1_S24_F1114', 'L1_S24_F1187', 'L2_S27_F3192', 'L1_S24_F1523', 'L3_S29_F3475', 'L1_S24_F1675', 'L1_S25_F2141', 'L2_S28_F3224', 'L1_S24_F1278', 'L1_S24_F1525', 'L3_S47_F4141', 'L1_S25_F3013', 'L1_S24_F1585', 'L2_S27_F3131', 'L2_S26_F3042', 'L1_S25_F2099', 'L3_S49_F4217', 'L1_S24_F1559', 'L3_S32_F3854', 'L3_S29_F3320', 'L1_S25_F1981', 'L1_S25_F1852', 'L1_S24_F1584', 'L1_S24_F1510', 'L1_S24_F695', 'L1_S25_F1903', 'L1_S24_F1537', 'L2_S26_F3099', 'L3_S29_F3317', 'L1_S24_F675', 'L1_S24_F819', 'L3_S35_F3899', 'L1_S25_F2802']
36


In [ ]:
#--------------------------------------------------last battle! ------------------------------------------------------------

# select categorical data based on numeric and date data ()


In [6]:
num_index=['Id', 'L3_S30_F3754', 'L3_S36_F3920', 'L3_S33_F3857', 'L3_S33_F3859', 'L3_S30_F3744', 'L0_S0_F20', 'L3_S33_F3865',
        'L3_S30_F3759', 'L3_S30_F3749', 'L3_S30_F3809', 'L0_S1_F28', 'L3_S29_F3479', 'L3_S30_F3774', 'L3_S30_F3554', 
        'L3_S30_F3804', 'L3_S30_F3769', 'L3_S29_F3339', 'L3_S30_F3829', 'L3_S30_F3574', 'L3_S32_F3850', 'L1_S24_F1723', 
        'L3_S29_F3407', 'L3_S29_F3345', 'L3_S29_F3348', 'L3_S29_F3351', 'L3_S38_F3952', 'L3_S35_F3889', 'L3_S29_F3327',
        'L3_S30_F3604', 'L3_S38_F3960', 'L3_S29_F3333', 'L3_S30_F3704', 'L3_S30_F3494', 'L3_S35_F3896', 'L3_S29_F3324',
        'L3_S29_F3373', 'L3_S30_F3609', 'L3_S29_F3379', 'L3_S30_F3639', 'L0_S1_F24', 'L2_S26_F3106', 'L3_S29_F3321', 
        'L3_S38_F3956', 'L0_S0_F16', 'L3_S29_F3336', 'L3_S30_F3534', 'L1_S24_F1846', 'L3_S29_F3427','L1_S24_F1604',
        'L1_S24_F1695', 'L3_S33_F3855','L1_S24_F1632']

In [7]:
d_col=['Id','L3_S36_D3919', 'L3_S38_D3953', 'L3_S33_D3856', 'L3_S35_D3886', 'L3_S29_D3474', 'L2_S26_D3037', 'L0_S3_D70', 
    'L3_S34_D3875', 'L0_S7_D137', 'L0_S10_D216', 'L0_S4_D106', 'L0_S9_D152', 'L2_S27_D3130', 'L0_S5_D115', 'L0_S2_D34', 
    'L0_S11_D280', 'L0_S17_D432', 'L1_S24_D1765', 'L3_S32_D3852', 'L0_S6_D120','L3_S29_D3316','L0_S0_D1','L3_S30_D3496',
       'L3_S30_D3506','L3_S30_D3501', 'L3_S30_D3516','L3_S30_D3511']

In [10]:
orgcol=list(set(num_index+d_col))

80

In [2]:
def ctg_importance(X_tr, response,orgcol):

    grid = {#'max_depth':10,
    #'booster': 'gbtree',
    'objective': 'binary:logistic',
    'subsample': 0.8,
    'colsample_bytree':1,
    'learning_rate': 0.12,
    'max_depth': 10,
    'seed': 2016,
    'silent': 0,
    'n_estimators': 200}

    clf_ctg = xgb.XGBClassifier()
    clf_ctg.set_params(**grid)

    clf_ctg.fit(X_tr,response.iloc[:,0])
    
    print("#########################training end#############################")
    
    column_name=X_tr.columns
    
    # get feature importance
    importance=clf_ctg.feature_importances_
    # sort index by importance value 
    importance_s=sorted([(column_name[index],value) for index, value in enumerate(importance.tolist())],
                        key=lambda x: x[1],reverse = True)
    # filter
    
    #topindex=[index for (index,val) in importance_s][0:30]
    print("------------------------------------------------------------------------------------")
    print([(name,val) for (name,val) in importance_s if val>0])
    print("------------------------------------------------------------------------------------")
    important_feature_name=[name for (name,val) in importance_s if val>0 and name not in orgcol]
    # get column name(full dummy name)
    
    mid=[ele.split('_')[:3] for ele in important_feature_name if '_' in ele]    
    # get original name
    imp_col_name=[]
        
    for ele in mid:
        if '_'.join(ele) in imp_col_name:
            continue
        else:
            imp_col_name.append('_'.join(ele))
            
    return imp_col_name

In [3]:
def readchunk(path_ctg, path_num, path_date):
    num_index=['Id', 'L3_S30_F3754', 'L3_S36_F3920', 'L3_S33_F3857', 'L3_S33_F3859', 'L3_S30_F3744', 'L0_S0_F20', 'L3_S33_F3865',
        'L3_S30_F3759', 'L3_S30_F3749', 'L3_S30_F3809', 'L0_S1_F28', 'L3_S29_F3479', 'L3_S30_F3774', 'L3_S30_F3554', 
        'L3_S30_F3804', 'L3_S30_F3769', 'L3_S29_F3339', 'L3_S30_F3829', 'L3_S30_F3574', 'L3_S32_F3850', 'L1_S24_F1723', 
        'L3_S29_F3407', 'L3_S29_F3345', 'L3_S29_F3348', 'L3_S29_F3351', 'L3_S38_F3952', 'L3_S35_F3889', 'L3_S29_F3327',
        'L3_S30_F3604', 'L3_S38_F3960', 'L3_S29_F3333', 'L3_S30_F3704', 'L3_S30_F3494', 'L3_S35_F3896', 'L3_S29_F3324',
        'L3_S29_F3373', 'L3_S30_F3609', 'L3_S29_F3379', 'L3_S30_F3639', 'L0_S1_F24', 'L2_S26_F3106', 'L3_S29_F3321', 
        'L3_S38_F3956', 'L0_S0_F16', 'L3_S29_F3336', 'L3_S30_F3534', 'L1_S24_F1846', 'L3_S29_F3427','L1_S24_F1604',
        'L1_S24_F1695', 'L3_S33_F3855','L1_S24_F1632']
        
    d_col=['Id','L3_S36_D3919', 'L3_S38_D3953', 'L3_S33_D3856', 'L3_S35_D3886', 'L3_S29_D3474', 'L2_S26_D3037', 'L0_S3_D70', 
    'L3_S34_D3875', 'L0_S7_D137', 'L0_S10_D216', 'L0_S4_D106', 'L0_S9_D152', 'L2_S27_D3130', 'L0_S5_D115', 'L0_S2_D34', 
    'L0_S11_D280', 'L0_S17_D432', 'L1_S24_D1765', 'L3_S32_D3852', 'L0_S6_D120','L3_S29_D3316','L0_S0_D1','L3_S30_D3496',
       'L3_S30_D3506','L3_S30_D3501', 'L3_S30_D3516','L3_S30_D3511']
    
    orgcol=list(set(num_index+d_col))
    
    size=200000
    for ctg, num, date, res in zip(pd.read_csv(path_ctg, chunksize=size), 
                       pd.read_csv(path_num, usecols=num_index, chunksize=size),
                       pd.read_csv(path_date, usecols=d_col, chunksize=size),
                       pd.read_csv(path_num, usecols=['Response'], chunksize=size)):
        
        fixed=pd.merge(num,date,on='Id')
        train_ctg=ctg.dropna(axis=1,how='all')
        pd_ctg=pd.get_dummies(train_ctg)
        
        train=pd.merge(fixed, pd_ctg, on='Id')
        print("####################data ready###################")
        cols=ctg_importance(train, res,orgcol)
        print(cols)

In [4]:
def main():
    path_ctg='Y://Data//Kaggle//bosch//train_categorical//train_categorical.csv'
    path_num='Y://Data//Kaggle//bosch//train_numeric//train_numeric.csv'
    path_date='Y://Data//Kaggle//bosch//train_date//train_date.csv'

    readchunk(path_ctg, path_num,path_date)

In [5]:
main()

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,44,45,46,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,324

####################data ready###################
#########################training end#############################
------------------------------------------------------------------------------------
[('Id', 0.04704039916396141), ('L3_S33_F3857', 0.03294448181986809), ('L3_S33_F3859', 0.029920069500803947), ('L3_S30_F3759', 0.02603154070675373), ('L3_S29_F3327', 0.02462734840810299), ('L3_S30_F3774', 0.022791098803281784), ('L3_S33_F3865', 0.022143011912703514), ('L3_S30_F3809', 0.022089004516601562), ('L3_S30_F3754', 0.021872974932193756), ('L3_S30_F3744', 0.020576799288392067), ('L0_S1_F28', 0.020468784496188164), ('L0_S0_F20', 0.019928710535168648), ('L3_S30_F3554', 0.019928710535168648), ('L3_S30_F3749', 0.019604666158556938), ('L3_S30_F3804', 0.019442643970251083), ('L3_S29_F3479', 0.01928062178194523), ('L3_S36_F3920', 0.018848562613129616), ('L3_S30_F3494', 0.018470512703061104), ('L3_S30_F3769', 0.01798444613814354), ('L3_S29_F3336', 0.01771440915763378), ('L0_S1_F24', 0.0176

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,44,45,46,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285

####################data ready###################
#########################training end#############################
------------------------------------------------------------------------------------
[('Id', 0.046331025660037994), ('L3_S33_F3857', 0.0324898287653923), ('L3_S33_F3859', 0.03143325075507164), ('L3_S30_F3809', 0.02467113919556141), ('L3_S33_F3865', 0.02377304621040821), ('L3_S30_F3774', 0.023667389526963234), ('L3_S30_F3759', 0.02350890077650547), ('L3_S36_F3920', 0.022505151107907295), ('L3_S29_F3327', 0.02208251878619194), ('L3_S30_F3749', 0.021924031898379326), ('L3_S30_F3744', 0.0218183733522892), ('L0_S0_F20', 0.0207089651376009), ('L3_S30_F3554', 0.02033916302025318), ('L3_S30_F3804', 0.02012784592807293), ('L3_S30_F3754', 0.020022187381982803), ('L0_S1_F28', 0.01901843771338463), ('L3_S30_F3769', 0.01843731850385666), ('L3_S29_F3479', 0.01817317306995392), ('L0_S0_F16', 0.017961857840418816), ('L3_S29_F3333', 0.017697712406516075), ('L3_S29_F3336', 0.0174863971769

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,324,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,3

...

####################data ready###################
#########################training end#############################
------------------------------------------------------------------------------------
[('Id', 0.0508161224424839), ('L3_S33_F3857', 0.033067453652620316), ('L3_S33_F3859', 0.028207702562212944), ('L3_S33_F3865', 0.024615710601210594), ('L3_S30_F3754', 0.024457238614559174), ('L0_S0_F20', 0.023717712610960007), ('L3_S29_F3327', 0.023612065240740776), ('L3_S30_F3759', 0.023083830252289772), ('L3_S30_F3774', 0.022661242634058), ('L3_S36_F3920', 0.022555597126483917), ('L3_S30_F3749', 0.021076541393995285), ('L3_S30_F3809', 0.021023718640208244), ('L3_S30_F3804', 0.020020073279738426), ('L3_S29_F3479', 0.01922772079706192), ('L3_S30_F3744', 0.01917489804327488), ('L3_S30_F3554', 0.018805135041475296), ('L0_S1_F28', 0.01795995980501175), ('L3_S30_F3574', 0.017590196803212166), ('L0_S0_F16', 0.017484549432992935), ('L3_S29_F3336', 0.017220431938767433), ('L3_S30_F3494', 0.01679

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,324,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,3

####################data ready###################
#########################training end#############################
------------------------------------------------------------------------------------
[('Id', 0.048243314027786255), ('L3_S33_F3859', 0.031515467911958694), ('L3_S33_F3857', 0.03093864768743515), ('L3_S30_F3754', 0.025432616472244263), ('L3_S30_F3774', 0.02490823343396187), ('L3_S29_F3327', 0.02396434172987938), ('L3_S33_F3865', 0.02380702644586563), ('L3_S36_F3920', 0.023439958691596985), ('L3_S30_F3809', 0.0219716839492321), ('L3_S30_F3744', 0.021709490567445755), ('L0_S0_F20', 0.02118510752916336), ('L3_S30_F3759', 0.02024121582508087), ('L3_S30_F3749', 0.019874148070812225), ('L3_S29_F3479', 0.019297325983643532), ('L3_S30_F3554', 0.019244886934757233), ('L3_S30_F3804', 0.018772941082715988), ('L3_S30_F3769', 0.01866806484758854), ('L0_S1_F28', 0.018353434279561043), ('L3_S30_F3574', 0.017986366525292397), ('L0_S0_F16', 0.01756685972213745), ('L3_S29_F3345', 0.0170949

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2885: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,188,189,190,191,192,193,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,2

####################data ready###################
#########################training end#############################
------------------------------------------------------------------------------------
[('Id', 0.0516594834625721), ('L3_S33_F3857', 0.032238204032182693), ('L3_S33_F3859', 0.03190239146351814), ('L3_S33_F3865', 0.026977108791470528), ('L3_S30_F3759', 0.025521911680698395), ('L3_S30_F3744', 0.022443611174821854), ('L3_S30_F3754', 0.022163765504956245), ('L3_S30_F3774', 0.022163765504956245), ('L0_S0_F20', 0.02193988859653473), ('L3_S30_F3809', 0.021492136642336845), ('L3_S29_F3327', 0.02132422849535942), ('L3_S30_F3554', 0.020204847678542137), ('L3_S36_F3920', 0.019813062623143196), ('L3_S30_F3494', 0.019477248191833496), ('L3_S30_F3749', 0.019477248191833496), ('L3_S29_F3348', 0.019085464999079704), ('L3_S30_F3804', 0.018022051081061363), ('L0_S1_F28', 0.01791011355817318), ('L3_S29_F3333', 0.017462361603975296), ('L3_S29_F3479', 0.017070576548576355), ('L3_S29_F3345', 0.

In [5]:
o=['Id','L2_S26_F3038', 'L3_S32_F3851', 'L1_S24_F1582', 'L1_S24_F1114', 'L1_S24_F1187', 'L2_S27_F3192', 'L1_S24_F1523',
          'L3_S29_F3475', 'L1_S24_F1675', 'L1_S25_F2141', 'L2_S28_F3224', 'L1_S24_F1278', 'L1_S24_F1525', 'L3_S47_F4141', 
          'L1_S25_F3013', 'L1_S24_F1585', 'L2_S27_F3131', 'L2_S26_F3042', 'L1_S25_F2099', 'L3_S49_F4217', 'L1_S24_F1559',
          'L3_S32_F3854', 'L3_S29_F3320', 'L1_S25_F1981', 'L1_S25_F1852', 'L1_S24_F1584', 'L1_S24_F1510', 'L1_S24_F695',
          'L1_S25_F1903', 'L1_S24_F1537', 'L2_S26_F3099', 'L3_S29_F3317', 'L1_S24_F675', 'L1_S24_F819', 'L3_S35_F3899', 
          'L1_S25_F2802','L1_S24_F1827']

In [6]:
p1=['L3_S32_F3854', 'L3_S49_F4217', 'L3_S35_F3899', 'L3_S32_F3851', 'L3_S29_F3317', 'L3_S29_F3475', 'L1_S24_F819',
    'L2_S27_F3192', 'L1_S25_F1852', 'L1_S25_F2141', 'L1_S25_F1903', 'L1_S25_F2099', 'L1_S24_F1137', 'L1_S25_F2597',
    'L1_S25_F2779', 'L3_S47_F4141', 'L1_S24_F675', 'L1_S24_F773', 'L2_S26_F3099', 'L3_S44_F4102', 'L1_S24_F705',
    'L1_S24_F910', 'L1_S24_F1114', 'L1_S24_F1136', 'L1_S24_F1187', 'L1_S24_F1278', 'L1_S24_F1525', 'L1_S24_F1530',
    'L1_S25_F1981', 'L1_S25_F2447', 'L1_S25_F2802', 'L2_S28_F3224', 'L2_S28_F3285']

In [7]:
p2=['L3_S32_F3854', 'L1_S24_F1278', 'L1_S25_F2141', 'L1_S25_F1981', 'L3_S29_F3317', 'L3_S35_F3902', 'L1_S25_F1903', 
    'L3_S29_F3475', 'L3_S32_F3851', 'L1_S24_F1525', 'L2_S28_F3285', 'L1_S25_F1852', 'L2_S27_F3192', 'L3_S47_F4141', 
    'L2_S28_F3224', 'L1_S24_F1114', 'L1_S24_F1137', 'L1_S24_F1156', 'L1_S24_F819', 'L1_S24_F1187', 'L1_S25_F2779',
    'L1_S25_F3013', 'L0_S2_F33', 'L1_S24_F773', 'L1_S24_F1136', 'L1_S24_F1510', 'L1_S24_F1559', 'L1_S25_F2099', 
    'L1_S25_F2802', 'L2_S26_F3099', 'L3_S44_F4102', 'L3_S49_F4207', 'L3_S49_F4217']

In [8]:
p3=['L3_S32_F3854', 'L1_S25_F1903', 'L3_S29_F3317', 'L3_S29_F3475', 'L1_S25_F2330', 'L1_S24_F1525', 'L1_S24_F1530',
    'L1_S25_F2141', 'L3_S47_F4141', 'L3_S49_F4217', 'L1_S24_F1114', 'L1_S25_F1852', 'L3_S32_F3851', 'L3_S35_F3899',
    'L1_S24_F819', 'L1_S24_F910', 'L1_S24_F1187', 'L1_S24_F1459', 'L1_S24_F1510', 'L1_S24_F1559', 'L1_S25_F1981', 
    'L1_S25_F2447', 'L2_S26_F3099', 'L2_S28_F3224', 'L3_S35_F3902', 'L1_S24_F675', 'L1_S24_F698', 'L1_S24_F705',
    'L1_S24_F710', 'L1_S24_F773', 'L1_S24_F1136', 'L1_S24_F1137', 'L1_S25_F2099', 'L1_S25_F2229', 'L1_S25_F2779', 
    'L1_S25_F2802', 'L1_S25_F2958', 'L1_S25_F3013']

In [9]:
p4=['L3_S32_F3854', 'L3_S32_F3851', 'L1_S25_F2779', 'L2_S28_F3224', 'L1_S25_F2099', 'L3_S29_F3317', 'L3_S29_F3475',
    'L1_S24_F1114', 'L1_S25_F1852', 'L1_S25_F1981', 'L1_S24_F1187', 'L2_S28_F3285', 'L1_S24_F1278', 'L1_S25_F2141', 
    'L1_S25_F2447', 'L3_S49_F4217', 'L1_S24_F1156', 'L1_S24_F703', 'L1_S24_F1530', 'L1_S24_F1559', 'L1_S25_F3013',
    'L2_S27_F3192', 'L3_S44_F4102', 'L3_S47_F4141', 'L1_S24_F819', 'L1_S24_F1137', 'L1_S24_F1459', 'L1_S24_F1510',
    'L1_S25_F1903', 'L1_S25_F2880', 'L1_S25_F2958', 'L3_S35_F3899']

In [10]:
p5=['L3_S32_F3854', 'L3_S29_F3475', 'L1_S24_F675', 'L1_S24_F1278', 'L3_S29_F3317', 'L1_S25_F2880', 'L1_S24_F910',
    'L1_S24_F1114', 'L3_S35_F3899', 'L3_S47_F4141', 'L1_S25_F2141', 'L1_S24_F710', 'L1_S25_F1981', 'L1_S25_F2099', 
    'L1_S25_F3013', 'L2_S28_F3285', 'L1_S24_F698', 'L1_S25_F1852', 'L1_S25_F2779', 'L2_S27_F3192', 'L3_S44_F4102', 
    'L3_S49_F4207', 'L1_S24_F695', 'L1_S24_F773', 'L1_S24_F819', 'L1_S24_F1156', 'L1_S24_F1185', 'L1_S24_F1187',
    'L1_S24_F1188', 'L1_S24_F1510', 'L1_S24_F1525', 'L1_S25_F1903', 'L1_S25_F2229', 'L1_S25_F2447', 'L2_S28_F3224',
    'L3_S32_F3851']

In [11]:
p6=['L3_S32_F3854', 'L1_S24_F819', 'L3_S29_F3317', 'L1_S24_F675', 'L3_S29_F3475', 'L1_S25_F2099', 'L1_S25_F1903',
    'L1_S25_F2141', 'L1_S25_F3013', 'L1_S25_F1852', 'L1_S24_F1187', 'L1_S24_F1523', 'L1_S24_F1137', 'L1_S24_F910', 
    'L1_S24_F1114', 'L1_S24_F1278', 'L1_S24_F1525', 'L1_S25_F2252', 'L1_S25_F2447', 'L2_S26_F3099', 'L2_S28_F3285', 
    'L1_S24_F698', 'L1_S24_F1140', 'L1_S24_F1156', 'L1_S24_F1530', 'L1_S24_F1585', 'L1_S25_F1981', 'L1_S25_F2779',
    'L2_S27_F3192', 'L3_S32_F3851', 'L3_S47_F4141']

In [12]:
kernel =['Id',
             'L1_S24_F1559', 'L3_S32_F3851',
             'L1_S24_F1827', 'L1_S24_F1582',
             'L3_S32_F3854', 'L1_S24_F1510',
             'L1_S24_F1525', 'L2_S26_F3099']

In [37]:
print(len(o))
#print(len(set(o)|set(kernel)))
print(len(p1))
print(len(set(o) & set(p1)))
print(len(big))
print(len(set(big)|set(kernel)))

38
33
22
55
58


In [15]:
big=list(set(p1)|set(p2)|set(p3)|set(p4)|set(p5)|set(p6))

In [19]:
small=list(set(p1)&set(p2)&set(p3)&set(p4)&set(p5)&set(p6))

In [16]:
combine=set(big)&set(o)
len(combine)

28

In [30]:
len(small)

14

In [32]:
#20 intersect among p and union kernel
list(set(small) | set(kernel))

['L1_S24_F1187',
 'L1_S25_F2779',
 'L1_S24_F819',
 'L2_S26_F3099',
 'L1_S25_F1903',
 'L3_S47_F4141',
 'L1_S25_F1981',
 'L3_S32_F3851',
 'L1_S24_F1559',
 'L1_S25_F2099',
 'Id',
 'L1_S24_F1525',
 'L1_S25_F1852',
 'L3_S29_F3475',
 'L1_S24_F1510',
 'L3_S32_F3854',
 'L1_S24_F1582',
 'L1_S24_F1827',
 'L1_S25_F2141',
 'L3_S29_F3317',
 'L1_S24_F1114']

In [23]:
#32

U10=list(set(p1[:10])|set(p2[:10])|set(p3[:10])|set(p4[:10])|set(p5[:10])|set(p6[:10]))
ctg30=list(set(U10) | set(kernel))
ctg30

['L1_S24_F1530',
 'L1_S24_F1827',
 'L1_S25_F2779',
 'L2_S27_F3192',
 'L1_S25_F1903',
 'L1_S25_F2141',
 'L1_S24_F819',
 'L1_S24_F1559',
 'L1_S25_F2330',
 'L1_S25_F1852',
 'L1_S25_F2099',
 'L3_S49_F4217',
 'L1_S24_F675',
 'L3_S35_F3902',
 'Id',
 'L3_S35_F3899',
 'L1_S25_F2880',
 'L3_S29_F3475',
 'L3_S29_F3317',
 'L1_S24_F910',
 'L1_S24_F1510',
 'L2_S28_F3224',
 'L1_S24_F1582',
 'L1_S24_F1114',
 'L2_S26_F3099',
 'L1_S25_F1981',
 'L3_S32_F3851',
 'L1_S25_F3013',
 'L1_S24_F1278',
 'L1_S24_F1525',
 'L3_S32_F3854',
 'L3_S47_F4141']

In [40]:
#39
U12=list(set(p1[:12])|set(p2[:12])|set(p3[:12])|set(p4[:12])|set(p5[:12])|set(p6[:12]))

ctg40=list(set(U12)|combine|set(kernel))

ctg40

['L1_S24_F1530',
 'L1_S25_F2779',
 'L1_S24_F1523',
 'L1_S24_F819',
 'L1_S24_F1559',
 'L1_S25_F1852',
 'Id',
 'L3_S35_F3902',
 'L1_S25_F2880',
 'L3_S29_F3475',
 'L3_S29_F3317',
 'L1_S24_F695',
 'L1_S24_F1510',
 'L2_S28_F3224',
 'L1_S24_F1114',
 'L1_S25_F2802',
 'L3_S32_F3851',
 'L1_S24_F1278',
 'L1_S24_F1525',
 'L3_S32_F3854',
 'L1_S24_F1585',
 'L1_S24_F1827',
 'L2_S27_F3192',
 'L1_S25_F1903',
 'L1_S25_F2141',
 'L1_S24_F1187',
 'L1_S25_F2330',
 'L1_S25_F2099',
 'L3_S49_F4217',
 'L1_S24_F675',
 'L3_S35_F3899',
 'L1_S24_F910',
 'L2_S26_F3099',
 'L1_S24_F1582',
 'L1_S24_F710',
 'L1_S25_F1981',
 'L2_S28_F3285',
 'L1_S25_F3013',
 'L3_S47_F4141']

In [34]:
kernel

['Id',
 'L1_S24_F1559',
 'L3_S32_F3851',
 'L1_S24_F1827',
 'L1_S24_F1582',
 'L3_S32_F3854',
 'L1_S24_F1510',
 'L1_S24_F1525',
 'L2_S26_F3099']

In [ ]:
#40

In [17]:
#50

U17=list(set(p1[:18])|set(p2[:18])|set(p3[:18])|set(p4[:18])|set(p5[:18])|set(p6[:18]))

ctg50=list(set(U17)|combine|set(kernel))
len(ctg50)

48

In [18]:
ctg50

['L1_S24_F1559',
 'L1_S24_F1510',
 'L1_S25_F2141',
 'L1_S25_F2330',
 'L1_S25_F1981',
 'L3_S35_F3899',
 'L2_S27_F3192',
 'L2_S26_F3099',
 'L3_S29_F3475',
 'L3_S32_F3854',
 'L3_S32_F3851',
 'L1_S24_F710',
 'L1_S24_F698',
 'L1_S25_F2802',
 'L1_S24_F1278',
 'L2_S28_F3285',
 'L1_S24_F819',
 'L3_S49_F4217',
 'L1_S25_F2880',
 'L1_S25_F2252',
 'L1_S24_F1827',
 'L1_S24_F1585',
 'L1_S25_F3013',
 'L1_S24_F1137',
 'L1_S24_F773',
 'L3_S29_F3317',
 'L1_S24_F1156',
 'L1_S24_F1525',
 'L1_S24_F1530',
 'L1_S24_F703',
 'L3_S47_F4141',
 'Id',
 'L1_S24_F1523',
 'L1_S24_F695',
 'L1_S25_F2099',
 'L1_S24_F1187',
 'L1_S25_F2779',
 'L1_S24_F675',
 'L1_S25_F2597',
 'L1_S24_F1114',
 'L1_S24_F1582',
 'L1_S24_F910',
 'L1_S25_F1852',
 'L2_S28_F3224',
 'L1_S25_F2447',
 'L1_S25_F1903',
 'L3_S35_F3902',
 'L1_S24_F1459']